In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)

In [2]:
#Abrir o site da busca jurídica:
import os

caminho = os.getcwd()
arquivo_site = caminho + r'\Arquivos - Consulta Juridica\index.html'
navegador.get(arquivo_site)


In [3]:
#Importar a base de dados a serem consultados:
import pandas as pd

tabela_processos = pd.read_excel(r'Arquivos - Consulta Juridica\Processos.xlsx')
display(tabela_processos)

,Nome,Advogado,Processo,Cidade,Status
0,Lira,Alon Lawyer,PC6592,Distrito Federal,NaN
1,João,Lawyer Alon,EB3792,Rio de Janeiro,NaN
2,Amanda,Amanda mesmo,MM1043,Rio de Janeiro,NaN
3,Carol,Amanda,PC5197,São Paulo,NaN


In [4]:
from selenium.webdriver import ActionChains
import time

for linha in tabela_processos.index:
    #Abrir a lista de cidades:
    botao = navegador.find_element(By.CLASS_NAME, 'dropdown-menu')
    ActionChains(navegador).move_to_element(botao).perform()
    
    #Selecionar a cidade:
    cidade = tabela_processos.loc[linha, 'Cidade']
    navegador.find_element(By.PARTIAL_LINK_TEXT, cidade).click()
    
    #Mudar para a nova aba:
    nova_aba = navegador.window_handles[1]
    navegador.switch_to.window(nova_aba)
    
    #Preencher o formulário com os dados de busca:
    navegador.find_element(By.ID, 'nome').send_keys(tabela_processos.loc[linha, 'Nome'])
    navegador.find_element(By.ID, 'advogado').send_keys(tabela_processos.loc[linha, 'Advogado'])
    navegador.find_element(By.ID, 'numero').send_keys(tabela_processos.loc[linha, 'Processo'])

    #Clicar em pesquisar:
    navegador.find_element(By.CLASS_NAME, 'registerbtn').click()
    
    #Confirmar a pesquisa:
    alerta = navegador.switch_to.alert
    alerta.accept()
    
    #Esperar o resultado da pesquisa e agir de acordo com o resultado:
    while True:
        try:
            alerta = navegador.switch_to.alert
            break
        except:
            time.sleep(1)
    texto_alerta = alerta.text

    if 'Processo encontrado com sucesso' in texto_alerta:
        alerta.accept()
        tabela_processos.loc[linha, 'Status'] = 'Encontrado'
    else:
        tabela_processos.loc[linha, 'Status'] = 'Não encontrado'
        alerta.accept()
    
    navegador.close()

    #Mudar para aba inicial:
    aba_inicial = navegador.window_handles[0]
    navegador.switch_to.window(aba_inicial)

navegador.quit()

In [5]:
display(tabela_processos)
tabela_processos.to_excel(r'Arquivos - Consulta Juridica\Processos Atualizado.xlsx')

,Nome,Advogado,Processo,Cidade,Status
0,Lira,Alon Lawyer,PC6592,Distrito Federal,Encontrado
1,João,Lawyer Alon,EB3792,Rio de Janeiro,Encontrado
2,Amanda,Amanda mesmo,MM1043,Rio de Janeiro,Não encontrado
3,Carol,Amanda,PC5197,São Paulo,Não encontrado
